In [1]:
import pandas as pd
import os
import datetime
import numpy as np
from tqdm import tqdm

game_ids.xlsx랑 collected_data가 있어야 돌아감.

In [2]:
PARTICIPANT_NUMBER_OF_A_TEAM = 5
participant_ids_by_role = []
DETAIL_PATH = "../data/collected_data/"

In [ ]:
def getstats(gameId, number):
    pass

In [4]:
strcolumn_dict = {"tournamentId":"str", "matchId":"str", "gameId":"str", "patch":"str"}
temp1 = [f"esportsPlayerId_{i}" for i in range(10)]
temp2 = ["str" for i in range(10)]
esportsPlayerId_type_dict = dict(zip(temp1, temp2))
strcolumn_dict.update(esportsPlayerId_type_dict)
game_ids_df = pd.read_excel("../data/game_ids.xlsx", dtype=strcolumn_dict, index_col = 0)
game_ids_df["startTime(match)"] = game_ids_df["startTime(match)"].apply(lambda x : datetime.datetime.strptime(x[:19], "%Y-%m-%dT%H:%M:%S"))

game_ids_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8023 entries, 0 to 8022
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         8023 non-null   datetime64[ns]
 1   tournamentId             8023 non-null   object        
 2   blockName                8023 non-null   object        
 3   leagueName               8023 non-null   object        
 4   matchId                  8023 non-null   object        
 5   bestof                   8023 non-null   int64         
 6   blueteam_name            8023 non-null   object        
 7   blueteam_code            8023 non-null   object        
 8   blueteam_win             8023 non-null   int64         
 9   redteam_name             8023 non-null   object        
 10  redteam_code             8023 non-null   object        
 11  redteam_win              8023 non-null   int64         
 12  gameNumberInAMatch       8023 non-null 

In [5]:
game_ids_df.head()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,esportsPlayerId_1,esportsPlayerId_2,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9,winner_side
0,2024-05-01 08:00:00,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,101389713966873995,98767991798919851,108369186353082038,106625308523122120,104573202011010010,101971954398227333,98767991764579452,98767975924333750,99566406444168246,Red
1,2024-05-01 08:00:00,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,101389713966873995,98767991798919851,108369186353082038,106625308523122120,104573202011010010,101971954398227333,98767991764579452,98767975924333750,99566406444168246,Blue
2,2024-05-01 08:00:00,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,101971954398227333,98767991764579452,98767975924333750,99566406444168246,99322214662601038,101389713966873995,98767991798919851,108369186353082038,106625308523122120,Red
3,2024-05-01 11:00:00,112099354464733424,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,...,105320682452092471,98767991747728851,103495716775975785,103495716561790834,105516185566739968,99566408350815597,102825747663199158,106276221575238438,105506922297942200,Blue
4,2024-05-01 11:00:00,112099354464733424,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,...,99566408350815597,102825747663199158,106276221575238438,105506922297942200,105320680474347057,105320682452092471,98767991747728851,103495716775975785,103495716561790834,Red


In [11]:
strcolumn_dict_of_collected_datas = {"gameId":"str", "esportsTeamId_Blue":"str", "esportsTeamId_Red":"str"}
strcolumn_dict_of_collected_datas.update(esportsPlayerId_type_dict)
last_row_of_collected_datas_df = pd.read_excel("../data/last_row_of_collected_datas.xlsx", dtype = strcolumn_dict_of_collected_datas)
last_row_of_collected_datas_df.info(verbose=True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8036 entries, 0 to 8035
Data columns (total 166 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    gameId                 8036 non-null   object 
 1    duration               8036 non-null   int64  
 2    esportsTeamId_Blue     8036 non-null   object 
 3    esportsTeamId_Red      8036 non-null   object 
 4    blue_totalGold         8036 non-null   int64  
 5    blue_inhibitors        8036 non-null   int64  
 6    blue_towers            8036 non-null   int64  
 7    blue_barons            8036 non-null   int64  
 8    blue_totalKills        8036 non-null   int64  
 9    blue_dragons_count     8036 non-null   int64  
 10   red_totalGold          8036 non-null   int64  
 11   red_inhibitors         8036 non-null   int64  
 12   red_towers             8036 non-null   int64  
 13   red_barons             8036 non-null   int64  
 14   red_totalKills         8036 non-null  

In [25]:
mapping = last_row_of_collected_datas_df.set_index('gameId')['esportsTeamId_Blue']
game_ids_df["esportsTeamId_Blue"] = game_ids_df["gameId"].map(mapping)
mapping = last_row_of_collected_datas_df.set_index('gameId')['esportsTeamId_Red']
game_ids_df["esportsTeamId_Red"] = game_ids_df["gameId"].map(mapping)

In [26]:
game_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8023 entries, 0 to 8022
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         8023 non-null   datetime64[ns]
 1   tournamentId             8023 non-null   object        
 2   blockName                8023 non-null   object        
 3   leagueName               8023 non-null   object        
 4   matchId                  8023 non-null   object        
 5   bestof                   8023 non-null   int64         
 6   blueteam_name            8023 non-null   object        
 7   blueteam_code            8023 non-null   object        
 8   blueteam_win             8023 non-null   int64         
 9   redteam_name             8023 non-null   object        
 10  redteam_code             8023 non-null   object        
 11  redteam_win              8023 non-null   int64         
 12  gameNumberInAMatch       8023 non-null 

In [27]:
game_ids_df = game_ids_df.loc[pd.isna(game_ids_df["esportsTeamId_Blue"]) == False]

In [28]:
game_ids_df.to_excel("../data/game_ids.xlsx")

여기까지는 game_ids에 esportsteamId를 붙이는 코드였음. 집와서 gameId다시읽을때 str타입으로 읽는 코드 추가할 것. strcolumn_dict에 넣으면 됨.

In [ ]:
game_ids_df_sorted = game_ids_df.sort_values(by="startTime(match)", ascending=False) # 최근 경기가 위로 오도록 내림차순 정렬
game_ids_df_sorted.head()

In [19]:
mapping = last_row_of_collected_datas_df.set_index('gameId')['esportsTeamId_Blue']
game_ids_df_sorted["esportsTeamId_Blue"] = game_ids_df_sorted["gameId"].map(mapping)
mapping = last_row_of_collected_datas_df.set_index('gameId')['esportsTeamId_Red']
game_ids_df_sorted["esportsTeamId_Red"] = game_ids_df_sorted["gameId"].map(mapping)
#game_ids_df_sorted["esportsTeamId_Blue"] = game_ids_df_sorted.apply(lambda row : last_row_of_collected_datas_df.loc[last_row_of_collected_datas_df["gameId"] == row["gameId"]]["esportsTeamId_Blue"], axis = 1)
#game_ids_df_sorted["esportsTeamId_Red"] = game_ids_df_sorted.apply(lambda row : last_row_of_collected_datas_df.loc[last_row_of_collected_datas_df["gameId"] == row["gameId"]]["esportsTeamId_Red"], axis = 1)
game_ids_df_sorted.tail()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9,winner_side,esportsTeamId_Blue,esportsTeamId_Red
884,2021-06-05 02:00:00,105658534671026792,Week 1,LCS,105658534675876677,1,FlyQuest,FLY,1,CLG,...,103461966875625344,100719501156073216,101389713973624205,98767975966749981,98767991777659617,98926509745704018,98767991804151434,Blue,98926509892121852,98926509884398584
883,2021-06-05 01:00:00,105658534671026792,Week 1,LCS,105658534675876651,1,100 Thieves,100,0,Immortals Progressive,...,99566406314527111,98926509793730393,103478281372021286,99322214660829968,98926509877984406,99566406296347493,98767975935156907,Red,99294153828264740,98767991930907107
882,2021-06-05 00:00:00,105658534671026792,Week 1,LCS,105658534675876665,1,Golden Guardians,GG,1,Cloud9,...,98926509773553311,104253650601859992,102206337829139149,99101098218071815,98767975948505333,98767975937423972,99101098208825497,Blue,99294153824386385,98767991877340524
881,2021-06-04 23:00:00,105658534671026792,Week 1,LCS,105658534675876687,1,Evil Geniuses LG,EG,0,Dignitas,...,105913361921383562,98767991788229386,101383793881527569,98926509797315114,101383793878119688,101383793885656347,98926509759999764,Red,103461966951059521,98926509883054987
880,2021-06-04 22:00:00,105658534671026792,Week 1,LCS,105658534675942263,1,Team Liquid Honda,TL,0,TSM,...,101383793877071109,98767991789638893,98767991783220297,99101098220080545,98767991776374577,99322400853596994,98767975903712783,Red,98926509885559666,98767991860392497


In [20]:
game_ids_df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8023 entries, 4020 to 880
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         8023 non-null   datetime64[ns]
 1   tournamentId             8023 non-null   object        
 2   blockName                8023 non-null   object        
 3   leagueName               8023 non-null   object        
 4   matchId                  8023 non-null   object        
 5   bestof                   8023 non-null   int64         
 6   blueteam_name            8023 non-null   object        
 7   blueteam_code            8023 non-null   object        
 8   blueteam_win             8023 non-null   int64         
 9   redteam_name             8023 non-null   object        
 10  redteam_code             8023 non-null   object        
 11  redteam_win              8023 non-null   int64         
 12  gameNumberInAMatch       8023 non-nul

In [23]:
game_ids_df_sorted.loc[pd.isna(game_ids_df_sorted["esportsTeamId_Blue"]) == True]["gameId"]

2577    111561337007305539
7067    106269875363891916
Name: gameId, dtype: object

2개는 _toshort로 표시한 그 2경기인걸로 확인

In [5]:
i = 0
for i in range(PARTICIPANT_NUMBER_OF_A_TEAM):       # 탑 정글 미드 원딜 서폿 이렇게..
    game_ids_with_playerId_blue = game_ids_with_picks_df[pd.isna(game_ids_with_picks_df[f"esportsPlayerId_{i}"]) == False]                                  # 블루팀(0~5)에서 서머너ID가 있는 것만
    game_ids_with_playerId_red = game_ids_with_picks_df[pd.isna(game_ids_with_picks_df[f"esportsPlayerId_{i + PARTICIPANT_NUMBER_OF_A_TEAM}"]) == False]    # 레드팀(5~9)에서 서머너ID가 있는 것만
    participant_ids_by_role.append(list(set(pd.concat([game_ids_with_playerId_blue[f"esportsPlayerId_{i}"],game_ids_with_playerId_red[f"esportsPlayerId_{i + PARTICIPANT_NUMBER_OF_A_TEAM}"]], ignore_index=True))))

print([len(i) for i in participant_ids_by_role])
print(participant_ids_by_role[0])


[346, 370, 368, 355, 394]
['101383792831678607', '104573231995499518', '98767991787052097', '107569568015156338', '101799086969700514', '106466536685100550', '105957589636391222', '107705430994631113', '101383792855140515', '106105043525540824', '104251865849200240', '105709413113050046', '107614697432935001', '99566408542840078', '101796918394601598', '107568579494407911', '99566404541748447', '98767975938371599', '109675627129279585', '98926509821954141', '99566408210057665', '109675500511180816', '107568586413146262', '105515229738531937', '111720165230650519', '107688960045460307', '100725844987233506', '109828407662355333', '98767975933568904', '107251528271309568', '101388913291808185', '105581684606195509', '107647477264255840', '100164952147744283', '104266795407626462', '108316132748778567', '108553450969769750', '107705432370779918', '99566408307919824', '107251646222113259', '100131567589299245', '105709428644866635', '111934286166932665', '107635905118503535', '107642475024

아 이제 각 포지션별 플레이어ID 리스트 뽑았고...

In [6]:
game_ids_df_sorted = game_ids_with_picks_df.sort_values(by="startTime(match)", ascending=False) # 최근 경기가 위로 오도록 내림차순 정렬
game_ids_df_sorted.head()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,esportsPlayerId_0,esportsPlayerId_1,esportsPlayerId_2,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9
4059,2024-05-28 10:00:00,112439909522772195,Week 1,LCO,112439909523296569,2,ION Global Esports,ION,2,Kanga Esports,...,111720042622049047,112477437508207101,111720045287922459,111720027901143523,111720051264281664,110462937178590834,111720122152475827,112477352770135560,112477356024384776,112477373979171280
4058,2024-05-28 10:00:00,112439909522772195,Week 1,LCO,112439909523296569,2,ION Global Esports,ION,2,Kanga Esports,...,111720042622049047,112477437508207101,111720045287922459,111720027901143523,111720051264281664,110462937178590834,111720122152475827,112477352770135560,112477356024384776,112477373979171280
4057,2024-05-28 08:00:00,112439909522772195,Week 1,LCO,112439909523296494,2,Fury,FRY,0,Team Bliss,...,105647956818593679,111720087560530528,110462943397555237,108604888219092306,111758035988327173,100160799381721105,104668539132702977,101383792849832093,103643243250697054,101383792831678607
4056,2024-05-28 08:00:00,112439909522772195,Week 1,LCO,112439909523296494,2,Fury,FRY,0,Team Bliss,...,105647956818593679,111720087560530528,110462943397555237,108604888219092306,111758035988327173,100160799381721105,104668539132702977,101383792849832093,103643243250697054,101383792831678607
4054,2024-05-27 10:00:00,112439909522772195,Week 1,LCO,112439909523296515,2,Antic Esports,ANC,2,Dire Wolves,...,107634897325128022,107634941727734818,105709436604717285,99566406296347493,102546029192465275,107647480732814180,112489008809745004,112488998397516384,102206314367722668,105714276869190870


In [9]:
file_list_in_detail = os.listdir(DETAIL_PATH)
for i, participant_ids in enumerate(participant_ids_by_role):     # 탑 정글 미드 원딜 서폿에 대해
    for participant_id in tqdm(participant_ids, total = len(participant_ids)):  # 모든 플레이어들에 대해
        filtered_df = game_ids_df_sorted[game_ids_df_sorted[f"esportsPlayerId_{i}"] == participant_id | \
            game_ids_df_sorted[f"esportsPlayerId_{i + PARTICIPANT_NUMBER_OF_A_TEAM}"] == participant_id].head(5)    # 블루팀 안에 있거나 레드팀 안에 있거나 상관없이 제일 최근 경기 5개를 가져온다.
        for j, row in filtered_df.iterrows():       # 와 3중 반복문 이게 맞나...?
            if row["gameId"] + ".xlsx" in file_list_in_detail:     # invalid말고 온전히 남아 있으면
                
                detail_df = pd.read_excel(DETAIL_PATH + row["gameId"] + ".xlsx")   # 데이터를 읽어와서
                #getstats(detail_df)
                
        

 17%|█▋        | 58/346 [04:12<20:51,  4.35s/it]


KeyboardInterrupt: 

이제 각 파일 들어가서 원하는 자료 뽑는 함수 만들어서 저기다 넣으면 됨...
getstats에는 gameId랑 participant_id가 몇번인지 그 번호를 넘길거고 그러기위해서는 몇번인지 찾아놔야한다